# NIRCam Imaging Depth

Here we use the JWST ETC Pandeia engine to calculate depths (5-sigma by default) for NIRCam imaging observations given your filters and exposure specifications. 

Default backgrounds may be used, or the background may be defined either by loading a file generated by the ETC or using the JWST Backgrounds Tool for a given RA, Dec and percentile as defined in the ETC web interface (Low=10%, Medium=50%, High=90%).

# Required Packages

In [1]:
from pandeia.engine.perform_calculation import perform_calculation
import numpy as np

# Optional Packages

In [2]:
from astropy import units as u  # convert magnitudes to nJy

from pandeia.engine.calc_utils import build_default_calc  # or alternatively, load ETC calculation parameters

# To define backgrounds, either need the JWST Backgrounds Tool,
#   or pyfits to load a file generated by the ETC web interface
from jwst_backgrounds import jbt  # To calculate background
from astropy.coordinates import SkyCoord # https://docs.astropy.org/en/stable/coordinates/
#
import astropy.io.fits as pyfits  # To load background

# Define ETC Calculation

Two options:  
1. Create default calculation from scratch
2. Load from ETC webpage calculation output

In [3]:
# ETC Pandeia inputs

if 1:
    imgr_data = build_default_calc('jwst','nircam','sw_imaging')
else:
    with open("nircam_imaging.json") as f:  # use a json file you have
        imgr_data = json.load(f)

# Photometric aperture and background sky annulus
aperture_radii = {'sw':0.08, 'lw':0.16, 'miri':0.3}  # (default 0.1" NIRCam, 0.3" MIRI)
imgr_data['strategy']['aperture_size'] = aperture_radii['sw']  # radius (default 0.1")
imgr_data['strategy']['sky_annulus'] = sky_annulus = 0.6, 0.99  # (default 0.22" - 0.4")

# Helper functions

In [4]:
# Converting flux density (nJy) to AB magnitudes

# without Astropy units
#from math import log10
#def nJytoAB(F_nJy):
#    return -2.5 * log10(F_nJy) + 31.4

# using Astropy units
def nJytoAB(F_nJy):
    return (F_nJy * u.nJy).to(u.ABmag).value

In [5]:
def NIRCam_Imaging_SNR(filt, mag, nexp, nint, ngroup, readmode, background=[]):
    global results, imgr_data

    if background != []:
        imgr_data['background'] = background

    # Assign filter and magnitude
    imgr_data['configuration']['instrument']['filter'] = filt.lower()
    imgr_data['scene'][0]['spectrum']['normalization']['norm_flux'] = mag
    imgr_data['scene'][0]['spectrum']['normalization']['norm_fluxunit'] = 'abmag'

    # Photometric aperture and background sky annulus
    lam = int(filt[1:4]) / 100.
    instrument = 'nircam'
    ch = 'sw lw'.split()[lam > 2.4]
    imgr_data['configuration']['instrument']['instrument'] = instrument
    imgr_data['configuration']['instrument']['aperture'] = ch
    imgr_data['configuration']['instrument']['mode'] = ch+'_imaging'
    imgr_data['strategy']['aperture_size'] = aperture_radii[ch]  # radius (default 0.1")
    imgr_data['strategy']['sky_annulus'] = sky_annulus  # (default 0.22" - 0.4")
    
    # Exposure specifications
    imgr_data['configuration']['detector']['nexp'] = nexp
    imgr_data['configuration']['detector']['nint'] = nint
    imgr_data['configuration']['detector']['ngroup'] = ngroup
    imgr_data['configuration']['detector']['readout_pattern'] = readmode.lower()
    
    # RUN CALCULATION
    results = perform_calculation(imgr_data)  
    SNR = results['scalar']['sn']
    total_exposure_time = results['scalar']['total_exposure_time']
    
    return SNR, total_exposure_time

In [6]:
def calculate_depth(filt, nexp, nint, ngroup, readmode, snr=5, background=[], tol=0.001):
    # Initial Guess:
    flux = 10  
    snr1 = 5
    diff = 1
    
    while diff > tol:
        flux = flux * snr / snr1
        mag = nJytoAB(flux)
        snr1, exptime = NIRCam_Imaging_SNR(filt, mag, nexp, nint, ngroup, readmode, background)
        diff = abs(snr1 - snr) / snr1

    return mag, exptime

In [7]:
def get_background(RA, Dec, percentile=50, lam=4.5):
    # percentiles used by ETC web interface: Low=10, Medium=50, High=90
    # evaluated at wavelength lam
    ra_dec_str = RA.replace(' ', ':') + ' ' + Dec.replace(' ',':')
    c = SkyCoord(ra_dec_str, unit=(u.hourangle, u.deg))
    ra_deg = c.ra.value
    dec_deg = c.dec.value
    # Note jbt uses lam and thresh for some calculations, but they don't affect the outputs used here
    bkg = jbt.background(ra_deg, dec_deg, wavelength=lam, thresh=1.1)
    calendar = bkg.bkg_data['calendar']
    bkg_vs_day_vs_lam = bkg.bkg_data['total_bg']
    bkg_lams = bkg.bkg_data['wave_array']
    ilam = bkg_lams.searchsorted(lam)
    bkg_vs_day = bkg_vs_day_vs_lam[:,ilam]
    ibkg_vs_day_sorted = bkg_vs_day.argsort()
    nbkg = len(ibkg_vs_day_sorted)
    ibkg_percentile = int(percentile / 100. * nbkg)
    ibkg_percentile = np.clip(ibkg_percentile, 0, nbkg-1)
    ibkg_day = ibkg_vs_day_sorted[ibkg_percentile]
    bkg_vs_lam = bkg_vs_day_vs_lam[ibkg_day]
    background = bkg_lams, bkg_vs_lam
    return background

# Example Calculations

Here we'll assume the default background defined by the ETC Pandeia engine.

In [8]:
filt = 'F444W'  # Filter
mag = 28.5    # Magnitude of source, if used as input
snr = 5     # Desired SNR, if used as input
nexp = 4    # Number of exposures (e.g., dithers)
nint = 1    # Number of integrations
ngroup = 9  # Number of groups
readmode = 'MEDIUM8'  # Readout pattern

In [9]:
SNR, total_exposure_time = NIRCam_Imaging_SNR(filt, mag, nexp, nint, ngroup, readmode)
SNR, total_exposure_time

(4.998581591812855, 3779.3430399999997)

In [10]:
mag, total_exposure_time = calculate_depth(filt, nexp, nint, ngroup, readmode, snr)
mag, total_exposure_time

(28.499724364817535, 3779.3430399999997)

In [11]:
# Same function call written differently
calculate_depth('F444W', 4, 1, 9, 'MEDIUM8')

(28.499724364817535, 3779.3430399999997)

In [12]:
# Same function call written differently
calculate_depth(filt='F444W', nexp=4, nint=1, ngroup=9, readmode='MEDIUM8', snr=5)

(28.499724364817535, 3779.3430399999997)

# Define the background

either by RA, Dec or by loading a file generated by the ETC web interface

In [13]:
if 1:
    # Calculate median background for a given field
    #target = 'ERS'  # well studied blank field
    RA  =  '03 32 42.397'
    Dec = '-27 42 7.93'
    background = get_background(RA, Dec, percentile=50)
else:
    # Alternatively, load file output by ETC
    bg_file = 'minzodi12_12052016.fits'
    bg_table = pyfits.getdata(bg_file)
    background = [bg_table['wavelength'],bg_table['background']] # wavelength in micron, SB in MJy/sr

In [14]:
calculate_depth(filt, nexp, nint, ngroup, readmode, snr, background=background)

(28.514596048429212, 3779.3430399999997)

# Calculate Depths for all filters

In [15]:
filts = 'F090W F115W F150W F200W F277W F356W F410M F444W'.split()
nexp = 4  # Number of exposures: 4 (e.g., 4 dithers)
nint = 1  # Number of integrations
ngroup = 9
readmode = 'MEDIUM8'
snr = 5

depths = {}
for filt in filts:
    mag, exptime = calculate_depth(filt, nexp, nint, ngroup, readmode, snr, background)
    depths[filt] = depth = mag
    line = '%s nexp=%d nint=%d ngroup=%d %s exptime=%5.1f mag=%5.2f SNR=%d' % (filt.ljust(6), nexp, nint, ngroup, readmode.ljust(7), exptime, depth, snr)
    print(line)

F090W  nexp=4 nint=1 ngroup=9 MEDIUM8 exptime=3779.3 mag=28.74 SNR=5
F115W  nexp=4 nint=1 ngroup=9 MEDIUM8 exptime=3779.3 mag=28.91 SNR=5
F150W  nexp=4 nint=1 ngroup=9 MEDIUM8 exptime=3779.3 mag=29.11 SNR=5
F200W  nexp=4 nint=1 ngroup=9 MEDIUM8 exptime=3779.3 mag=29.31 SNR=5
F277W  nexp=4 nint=1 ngroup=9 MEDIUM8 exptime=3779.3 mag=28.92 SNR=5
F356W  nexp=4 nint=1 ngroup=9 MEDIUM8 exptime=3779.3 mag=28.95 SNR=5
F410M  nexp=4 nint=1 ngroup=9 MEDIUM8 exptime=3779.3 mag=28.25 SNR=5
F444W  nexp=4 nint=1 ngroup=9 MEDIUM8 exptime=3779.3 mag=28.51 SNR=5


# Example exposure setup

NIRCam observes simultaneously in two filters at "short" wavelengths (< 2.4 microns) and "long" wavelengths (> 2.4 microns)

In [16]:
# Each line corresponds to one exposure setup with two filters observed simultaneously
exposures = """
F090W F444W MEDIUM8 9 1
F115W F410M MEDIUM8 9 1
F150W F356W MEDIUM8 6 1
F200W F277W MEDIUM8 6 1
""".split('\n')[1:-1]

nexp = 4  # Number of exposures: 4 (e.g., 4 dithers)

In [17]:
depths = {}
snr = 5
for exposure in exposures:
    filt1, filt2, readmode, ngroup, nint = exposure.lower().split()
    nint = int(nint)
    ngroup = int(ngroup)
    for filt in filt1, filt2:
        mag, exptime = calculate_depth(filt, nexp, nint, ngroup, readmode, snr, background)
        depths[filt] = depth = mag
        line = '%s %2d %2d %2d %s %7.1f  mag=%5.2f SNR=%d' % (filt.ljust(6), nexp, nint, ngroup, readmode.ljust(8), exptime, depth, snr)
        print(line)

f090w   4  1  9 medium8   3779.3  mag=28.74 SNR=5
f444w   4  1  9 medium8   3779.3  mag=28.51 SNR=5
f115w   4  1  9 medium8   3779.3  mag=28.91 SNR=5
f410m   4  1  9 medium8   3779.3  mag=28.25 SNR=5
f150w   4  1  6 medium8   2490.9  mag=28.83 SNR=5
f356w   4  1  6 medium8   2490.9  mag=28.71 SNR=5
f200w   4  1  6 medium8   2490.9  mag=29.02 SNR=5
f277w   4  1  6 medium8   2490.9  mag=28.68 SNR=5


# How much could depth vary depending on the date of the observation? 

Compare dates with lowest and highest possible backgrounds for the given RA, Dec. 

Note larger variations are found at longer wavelengths, such as with F444W.

In [18]:
# Percentile = 0: Day with lowest backgrounds. Yields best depth (highest magnitude)
background = get_background(RA, Dec, percentile=0)
calculate_depth(filt, nexp, nint, ngroup, readmode, snr, background)

(28.674773945591348, 2490.93064)

In [19]:
# Percentile = 100: Day with highest backgrounds. Yields worst depth (lowest magnitude)
background = get_background(RA, Dec, percentile=100)
calculate_depth(filt, nexp, nint, ngroup, readmode, snr, background)

(28.552944323681334, 2490.93064)